In [1]:
import pandas as pd
import datarobot as dr

dr.Client()

In [2]:
dr.Context.use_case

UseCase(id=66c8a66a6edd694c8ca75f20, name=Subscribers Churn, description=None, models=564, projects=12, datasets=28, notebooks=4, applications=0)

In [3]:
use_case = dr.Context.use_case
use_case

UseCase(id=66c8a66a6edd694c8ca75f20, name=Subscribers Churn, description=None, models=564, projects=12, datasets=28, notebooks=4, applications=0)

In [4]:
target = 'churned'
datetime_col = 'observation_date'

target_window = 60

In [6]:
drop_features = ['idsubscrip', 'stop_intent_date', 'idpermstartdate', 'last_new_start_date', 'recency_date']
drop_features += [
    'idsystem',
    'pubcode',
    'newsgroup',
    'publication_name',
    'market',
    'Top12_Growth_Market',
    'rfv_bin',
    'rfv_bin_score',
    'prev_rfv_bin_score',
    'rfv_bin_score_change',
    'rolling_31d_avg_rfv_bin_score',
    'rfv_bucket_movement_score',
    'recency_bucket',
    'stop_intents_last_7_days',
    'stop_intents_last_14_days',
]

In [7]:
datasets = dr.Dataset.list()
datasets

[Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20250205_trainset_1perc-2025-02-07T17:19:33.499Z', id='67a640a67200381a9991d65f'),
 Dataset(name='[valid] wk_training_20250120_test_idsubscrip Wrangled 2025-01-24 12-38-32', id='6793d02b9f1b46a9bbd7331b'),
 Dataset(name='wk_training_20250120_test_idsubscrip Wrangled 2025-01-24 12-29-00', id='6793ce001239a2955888a470'),
 Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20250120_test_idsubscrip-2025-01-24T17:12:11.215Z', id='6793c9ecdbf44b581688a3c5'),
 Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20250120_trainset_5perc-2025-01-23T20:18:58.302Z', id='6792a4331e904f2d07b10c55'),
 Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20241226_trainset_5perc-2024-12-27T01:39:30.285Z', id='676e055282bb13f1428e55c4'),
 Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20241223_trainset_5perc-2024-12-24T11:14:42.443Z', id='676a97a3c11685ba37340d7b'),
 Dataset(name='intention_to_stop_predictions_mid67461c3b

In [8]:
# train_dataset_name = 'gannett-datascience-DR_Vitalii-wk_stop_intent_training_current-2024-10-15T15:41:38.462Z'
# train_dataset_path = [x for x in datasets if x.name == train_dataset_name][0] #or by id

#byid
train_dataset_path = [x for x in dr.Dataset.list() if x.id == '67a640a67200381a9991d65f'][0] #or by id

train_dataset_path

Dataset(name='gannett-datascience-DR_Vitalii-wk_training_20250205_trainset_1perc-2025-02-07T17:19:33.499Z', id='67a640a67200381a9991d65f')

In [0]:
#

In [22]:
project = dr.Project.create_from_dataset(
    dataset_id=train_dataset_path.id, 
    project_name='_'.join(train_dataset_path.name.split('-')[3:-3]), 
    max_wait=60*60*24,
    use_case=use_case,
)


In [23]:
project.get_uri()


'https://app.datarobot.com/projects/67a648b75bfcc8d4c9a7d3c4/models'

In [21]:
# project= dr.Project.get('6792be89d7b608a4f52457c4')

In [24]:
settings = {}

settings["target"] = target  # Name of the target feature
settings["datetime_partition_column"] = datetime_col  # Date Column which is used to partition the data

settings["number_of_backtests"] = 1  # Number of backtests for the model to create
settings["gap_duration"] = f"P0Y0M{target_window}DT0H0M0S"  #60d Gap between training and validation folds
settings["validation_duration"] = "P0Y2M0DT0H0M0S"  #1M Duration of validation data
settings["disable_holdout"] = False  # disable holdout as we (should) use 'external'
# settings["holdout_duration"] = "P0Y1M0DT0H0M0S" #1M Duration of validation data

settings["worker_count"] = -1  # Number of modelling workers to use, -1 will use all available workers

In [25]:
#settings experiment 2: no holdout, 3 backtests

settings = {}

settings["target"] = target  # Name of the target feature
settings["datetime_partition_column"] = datetime_col  # Date Column which is used to partition the data

settings["number_of_backtests"] = 3  # Number of backtests for the model to create
settings["gap_duration"] = f"P0Y0M{target_window}DT0H0M0S"  #60d Gap between training and validation folds
settings["validation_duration"] = "P0Y2M0DT0H0M0S"  #1M Duration of validation data
settings["disable_holdout"] = True  # disable holdout as we (should) use 'external'
# settings["holdout_duration"] = "P0Y1M0DT0H0M0S" #1M Duration of validation data

settings["worker_count"] = -1  # Number of modelling workers to use, -1 will use all available workers

In [26]:
#TODO: add defaults
spec = dr.DatetimePartitioningSpecification(
    disable_holdout=settings["disable_holdout"],
    # holdout_duration=settings["holdout_duration"],
    datetime_partition_column=settings["datetime_partition_column"],
    number_of_backtests=settings["number_of_backtests"],
    gap_duration=settings["gap_duration"],
    validation_duration=settings.get("validation_duration"),
)

In [27]:
advanced_options = dr.AdvancedOptions(
    shap_only_mode = True,# True,
    autopilot_with_feature_discovery=True,
    run_leakage_removed_feature_list=True,
)

In [14]:
# ?dr.AdvancedOptions
#add incremental learning

In [28]:
project.get_featurelists()

[Featurelist(Raw Features), Featurelist(Informative Features)]

In [29]:
# Get informative features
informative_features = project.get_featurelist_by_name("Raw Features").features #Informative
predictors = [x for x in informative_features if (x not in [target, datetime_col, 'recency_date'] + drop_features) 
              and (datetime_col not in x)
              and (' (Year)' not in x) and (' (Day of Month)' not in x) and (' (Day of Week)' not in x) and (' (Week)' not in x) and (' (Month)' not in x) 
             ]
predictors

['days_since_last_new_start_date',
 'site_code',
 'tier',
 'ExactAge',
 'presence_of_young_adult',
 'lt_hs_grad',
 'hs_grad',
 'college_grad',
 'age_first_individual',
 'census_median_home_value',
 'children_present',
 'clerical',
 'craftperson',
 'e_hispanic_language_preference',
 'financial_pro',
 'home_assessed_value',
 'home_length_of_residence',
 'home_owner',
 'home_purchase_amt',
 'income_est_hh',
 'managerial',
 'married',
 'number_of_children',
 'mdu',
 'occupation',
 'owner_renter',
 'professional',
 'retired',
 'salesperson',
 'self_employed',
 'sfdu',
 'tech_pro',
 'recency_to_snapshot',
 'frequency',
 'volume',
 'modelscore',
 'recency_change',
 'frequency_change',
 'volume_change_pct',
 'modelscore_change_pct',
 'frequency_change_rate',
 'volume_change_rate',
 'rolling_31d_avg_frequency',
 'rolling_31d_avg_volume',
 'rolling_31d_avg_modelscore',
 'rolling_31d_std_frequency',
 'rolling_31d_std_volume',
 'rolling_31d_min_frequency',
 'rolling_31d_max_frequency',
 'rolling_3

In [30]:
actionable = [
    "unique_articles_90d",
    "unique_devices_90d",
    "unique_platforms_90d",
    "unique_sites_90d",
    "unique_sections_90d",
    "video_plays_90d",
    "page_views_90d",
    "first_30d_sessions",
    "first_30d_active_days",
    "first_30d_app_sessions",
    "first_30d_sections",
    "first_30d_video_plays",
    "first_30d_morning_days",
    "first_30d_weekend_days",
    "first_week_sessions",
    "first_30d_app_adoption_rate",
    "first_30d_engagement_ratio",
    "first_30d_enthusiasm_ratio",
    "first_30d_weekend_ratio",
    "first_30d_5andmore_sessions",
    "app_sessions_90d",
    "app_active_days_90d",
    "app_usage_ratio",
    "app_trend_ratio",
    "app_weekend_ratio",
    "app_articles_90d",
    "app_sections_90d",
    "app_video_plays_90d",
    "app_morning_days_90d",
    "app_weekend_days_90d",
    "app_sessions_30d",
    "primary_app_device",
    "primary_device_ratio",
    "unique_sources_90d",
    "unique_mediums_90d",
    "unique_referrers_90d",
    "direct_traffic_ratio",
    "primary_source_medium",
    "primary_source_medium_count",
    "primary_referrer",
    "primary_referrer_count",
    "direct_sessions_90d",
    "primary_source_ratio",
    "primary_referrer_ratio",
    "app_downloaded",
    "app_adoption_timing",
    "n_conversations",
    "full_conversation_text",
    "customer_message_count",
    "agent_message_count",
    "total_agent_speak_time",
    "total_customer_speak_time",
    "interaction_duration",
    "handle_time",
    "wait_time",
    "ring_duration",
    "talk_time",
    "hold_time",
    "acw_time",
    "queue",
    "location",
    "days_since_last_conversation",
    "days_with_push_notifications",
    "avg_daily_push_depth",
    "avg_daily_pushes",
    "max_daily_pushes",
    "total_breaking_news",
    "days_with_breaking_news",
    "active_hours_ratio_breaking_news",
    "weekday_ratio_breaking_news",
    "avg_pushes_last_14d",
    "recent_breaking_news_notifications",
    "breaking_news_std",
]

In [31]:
print(len(actionable))
actionable = [x for x in actionable if x in informative_features]
print(len(actionable))
actionable_featurelist = project.create_featurelist(name="actionable predictors", features=actionable)


73
72


In [32]:
modeling_featurelist = project.create_featurelist(name="selected predictors", features=predictors)


In [20]:
# modeling_featurelist = project.get_featurelist_by_name("selected")


In [34]:
project.analyze_and_model(
    target=settings["target"],
    mode=dr.enums.AUTOPILOT_MODE.COMPREHENSIVE,
    featurelist_id=modeling_featurelist.id,
    partitioning_method=spec,
    advanced_options=advanced_options,
    worker_count=settings["worker_count"],
    max_wait=20000,
)


Project(wk_training_20250205_trainset_1perc)

In [24]:
# project.analyze_and_model(
#     target=settings["target"],
#     mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,
#     featurelist_id=actionable_featurelist.id,
#     partitioning_method=spec,
#     advanced_options=advanced_options,
#     worker_count=settings["worker_count"],
#     max_wait=20000,
# )


/etc/system/kernel/.venv/lib64/python3.11/site-packages/datarobot/models/project.py:1655: PartitioningMethodWarning: The `partitioning_method` passed directly into `analyze_and_model` will be used, but                         `self.partitioning_method` is non-null. If you wish to instead use                         `self.partitioning_method`, call `set_partitioning_method` or                         `set_datetime_partitioning` on your project instance and do not pass a                         partitioning method directly to `analyze_and_model`.
  warnings.warn(


ClientError: 422 client error: {'message': 'Target churned was already selected.'}

In [0]:
project.wait_for_autopilot(timeout=None, verbosity=dr.VERBOSITY_LEVEL.SILENT)

In [2]:
from fire import Fire

In [5]:
f = Fire('67a6411ad968c600b0a7c751')

In [6]:
best_model_previous = f.get_best_models(metric='Weighted LogLoss').iloc[0]
best_model_previous


DatetimeModel('Dropout Additive Regression Trees Classifier  (15 leaves)')

In [41]:

# Kick off feature selection
best_model_post_fire = f.main_feature_selection(
    partition="validation",
    # start_featurelist_name='selected predictors +'
    # best_model_search_params={"sample_pct__lte": 65},
)



Request Feature Impact calculations


AsyncProcessUnsuccessfulError: The job did not complete successfully. Job Data: {'id': '56', 'projectId': '67a648b75bfcc8d4c9a7d3c4', 'status': 'ABORTED', 'jobType': 'featureImpact', 'isBlocked': False, 'url': 'https://app.datarobot.com/api/v2/projects/67a648b75bfcc8d4c9a7d3c4/jobs/56/', 'modelId': '67a66bf36bf0f970f4d61fa8', 'message': ''}